# Exercise 1: Agent Hello World

**Goal:** Create a simple AI agent with a custom weather tool using the OpenAI Agents SDK.

**Timebox:** 15 minutes

## What You'll Learn

- How to create an agent using the OpenAI Agents SDK
- How to define custom tool functions with `@function_tool`
- How to make real API calls (National Weather Service)
- How to run the agent with asyncio

## Setup

Import required libraries and load environment variables.

In [ ]:
%pip install --quiet openai-agents httpx python-dotenv nest-asyncio

import os
import asyncio
import httpx
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool

# Load environment variables
load_dotenv()

print("✅ Environment loaded")

## Step 1: Define a Weather Tool

Create a `get_weather_alerts()` function that calls the National Weather Service API.

**TODO:** Implement the function to fetch real weather alerts.

In [ ]:
@function_tool
async def get_weather_alerts(state: str) -> str:
    """
    Get current weather alerts for a US state from the National Weather Service.
    
    Args:
        state: Two-letter US state code (e.g., 'CA', 'NY', 'TX')
    
    Returns:
        String describing current weather alerts
    """
    # TODO: Make a real API call to the National Weather Service
    # API endpoint: https://api.weather.gov/alerts/active/area/{state}
    # Hint: Use async with httpx.AsyncClient() as client
    # Hint: await client.get(url, headers=headers, timeout=10.0)
    # Hint: Parse the JSON response and extract alert information
    # Hint: Handle cases where there are no active alerts
    
    pass

## Step 2: Create an Agent

Use the OpenAI Agents SDK to create an agent with instructions and tools.

**TODO:** Define the agent using the `Agent` class.

In [ ]:
# TODO: Create an agent with the Agents SDK
# agent = Agent(
#     name="Weather Agent",
#     instructions="You help users get weather alert information for US states",
#     tools=[get_weather_alerts]
# )

## Step 3: Verify OpenAI API Key

Ensure your OpenAI API key is configured.

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY is not set. Add it to your .env file.")

masked_key = f"{api_key[:4]}...{api_key[-4:]}" if len(api_key) >= 8 else "***"
print(f"✅ OPENAI_API_KEY detected: {masked_key}")

## Step 4: Run the Agent

Execute the agent with an example query.

In [ ]:
async def run_agent():
    query = "Are there any weather alerts for California?"
    print(f"Query: {query}\n")
    
    # TODO: Run the agent using Runner.run()
    # Hint: result = await Runner.run(agent, query)
    # TODO: Display the final output
    # Hint: print(f"\n🤖 Agent Response:\n{result.final_output}\n")
    
    print("⚠️  Implement the TODOs above!")
    print("   Check the solution notebook for reference")

# Jupyter-specific async handling (notebooks already have an event loop)
# In a regular Python script, you would use: asyncio.run(run_agent())
try:
    loop = asyncio.get_running_loop()
    import nest_asyncio
    nest_asyncio.apply()
    await run_agent()
except RuntimeError:
    asyncio.run(run_agent())

## Stretch Goal

- Add error handling for invalid state codes or API failures
- Modify the tool to get weather for a specific city by adding geocoding
- Try querying for multiple states in one question